In [1]:
import simulation_commander as simc
import grSim_Commands_pb2
import grSim_Packet_pb2
import time
import numpy as np
import math

client = simc.Client(ip='127.0.0.1', port=20011)

In [2]:
def prep_message_and_send(yellowTeam, id, kickspeedx, kickspeedz, veltangent, velnormal, velangular, spinner, wheelsspeed, wheel1, wheel2, wheel3, wheel4):
    packet = grSim_Packet_pb2.grSim_Packet()
    packet.commands.isteamyellow = yellowTeam
    packet.commands.timestamp = 0
    robot_command = grSim_Commands_pb2.grSim_Robot_Command()
    robot_command.id = id
    robot_command.kickspeedx = kickspeedx
    robot_command.kickspeedz = kickspeedz
    robot_command.veltangent = veltangent
    robot_command.velnormal = velnormal
    robot_command.velangular = velangular
    robot_command.spinner = spinner
    robot_command.wheelsspeed = wheelsspeed
    robot_command.wheel1 = wheel1   # front left
    robot_command.wheel2 = wheel2   # back left
    robot_command.wheel3 = wheel3   # back right
    robot_command.wheel4 = wheel4   # front right
    packet.commands.robot_commands.append(robot_command)
    
    while True:
        client.send(packet)
        time.sleep(0.01)

In [3]:


R = 1

######## Angulos das rodas em relação ao eixo x
######## 0 -> 30 graus front right
######## 1 -> 150 graus front left
######## 2 -> 225 graus back left
######## 3 -> 315 graus back right
wheel_alngles = [math.pi/6, (math.pi*5)/6, (math.pi*5)/4, (math.pi*7)/4]

######## Matriz Jacobiana
jacobian = np.matrix([[math.sin(wheel_alngles[0]), math.cos(wheel_alngles[0]), R],
                      [math.sin(wheel_alngles[1]), math.cos(wheel_alngles[1]), R],
                      [math.sin(wheel_alngles[2]), math.cos(wheel_alngles[2]), R],
                      [math.sin(wheel_alngles[3]), math.cos(wheel_alngles[3]), R],])

######## Eixo x, eixo y, rotação
velocities = np.matrix([[-1], [1], [0]])

In [4]:
wheels_acc = (1/R*jacobian*velocities).T.tolist()[0]
wheels_acc

[0.36602540378443876,
 -1.3660254037844386,
 -2.220446049250313e-16,
 1.414213562373095]

In [5]:
import grSim_Replacement_pb2

packet = grSim_Packet_pb2.grSim_Packet()
robot_replacement = grSim_Replacement_pb2.grSim_RobotReplacement()
robot_replacement.x = 0
robot_replacement.y = 4.5
robot_replacement.dir = 270
robot_replacement.id = 0
robot_replacement.yellowteam = 1
packet.replacement.robots.append(robot_replacement)


client.send(packet)
time.sleep(0.01)

In [6]:
prep_message_and_send(True, 0, 0, 0, 0, 0, 0, 0, 1, wheels_acc[1], wheels_acc[2], wheels_acc[3], wheels_acc[0])